In [2]:
import pandas as pd
import csv

#### Preprocess the_donald user data

In [5]:
#Load Data
col_types = {'author': str, 'body':str, 'subreddit':str, 'created_utc':int, 'score': int}
td_df = pd.read_csv('./data/user_histories/td_userhistdata_000000000001', dtype = col_types)
#td_df = td_df.append(pd.read_csv('./data/user_histories/td_userhistdata_000000000001', dtype = col_types), ignore_index=True)

ParserError: Error tokenizing data. C error: EOF inside string starting at line 740456

In [398]:
#Preprocess Data
td_df = td_df[td_df['subreddit'] == 'The_Donald'] #Filter by relevant subreddit
td_df = td_df[(td_df['created_utc'] >= 1454284800) & (td_df['created_utc'] < 1468540800)] #Filter by relevant time frame 

td_df = td_df.sort_values(['created_utc','author']).reset_index(drop=True) #Sort Data
td_df['body'] = td_df['body'].apply(lambda x: str(x)) #fix exceptions [was reading as NaN instead of string]

#Replace open parens with space (to account for hyperlink formatting on reddit)
td_df['body'] = td_df['body'].apply(lambda x: x.replace(r"(","  ")) 
td_df['body'] = td_df['body'].apply(lambda x: x.replace(r")","  ")) 

In [399]:
#Group data s.t. authors/bodies are grouped together
td_df = td_df.groupby('author')['body'].apply(lambda x: "%s" % ' '.join(x)).reset_index()
#This should result in 1 combined string per person (each person posts one "super post" made up of all their posts)

#Import and utilize a function from Jack's script which roughly sanitizes input strings
from fighting_words_py3 import basic_sanitize 
td_df['body'] = td_df['body'].apply(lambda x: basic_sanitize(x).split()) 

#Replace all strings which start with "http" with a marker for hyperlinks
td_df['body'] = td_df['body'].apply(lambda x: [i if i[0:4] != 'http' else '<HYPERLINK>' for i in x])
td_df.head(10)

,author,body
0,-nyx-,"[the, ruling, coalition, already, doesnt, have..."
1,0piat3,"[do, you, think, the, black, gun, is, scarier,..."
2,1000Clowns,"[if, that, happens, this, republican, since, 1..."
3,1000stomachcrunches,"[well, yeah, despite, being, the, best, and, b..."
4,1011011,"[if, thats, the, case, she, should, be, pullin..."
5,11234a3,"[yeesh, who, jerks, it, to, 3d, women, anymore..."
6,1337and0,"[aw, dang, i, shouldve, changed, ricky, to, re..."
7,138glv,"[god, dam, rekt, we, can, all, agree, induce, ..."
8,13inchpoop,"[women, are, always, the, ones, bitching, abou..."
9,1875coalminer,"[whoa, put, a, nsfw, tag, on, that, too, much,..."


In [401]:
#Export data
td_df.to_csv('./data/language_models/td_preproc_langmodel.csv', index=False)

#### Preprocess sandersforpresident language model base data

In [8]:
#Load Data
sfp_df = pd.read_csv('./data/user_histories/sfp_userhistdata_000000000000', dtype = col_types)
#sfp_df = sfp_df.append(pd.read_csv('./data/user_histories/sfp_userhistdata_000000000001', dtype = col_types), ignore_index=True)
sfp_df.head(10)

,author,body,subreddit,created_utc,score
0,ThomK,It's interesting to think about the theology b...,funny,1224801481,0
1,superiority,Downmodded for being on the CrimethInc. websit...,pics,1213772343,0
2,banjowashisnameo,Ghosts do not have data caps as far as I know,Showerthoughts,1447955469,1
3,rasheemo,you're welcome &lt;3,humor,1225832208,2
4,rasheemo,this post would have been fine without the unn...,reddit.com,1277522212,1
5,IBFT,That's what I'm using.,Nexus5,1431914223,1
6,hbgoddard,&gt;zoom in the googles\n\n&gt;googles,leagueoflegends,1432432333,3
7,miashaee,No idea as I do not know the full limits of wh...,DebateReligion,1421178891,1
8,ArchbishopHamster,This is the kind of quality shit I'm here for.,baseball,1444357974,4
9,Mikebyrneyadigg,The man was wounded laying on the ground and t...,worldnews,1458909012,5


In [10]:
#Preprocess Data
sfp_df = sfp_df.sort_values(['created_utc','subreddit','author']).reset_index(drop=True) #Sort Data
sfp_df['body'] = sfp_df['body'].apply(lambda x: str(x)) #fix exceptions [was reading as NaN instead of string]
sfp_df.head(10)


#Replace open parens with space (to account for hyperlink formatting on reddit)
#sfp_df['body'] = sfp_df['body'].apply(lambda x: x.replace(r"(","  ")) 
#sfp_df['body'] = sfp_df['body'].apply(lambda x: x.replace(r")","  ")) 

,author,body,subreddit,created_utc,score
0,crickets,That seems to me to be the inherent flaw with ...,reddit.com,1150350296,12
1,crickets,I suppose I should clarify what I view to be t...,reddit.com,1150417797,1
2,crickets,There is an outtake from this interview up on ...,reddit.com,1150842759,2
3,limukala,"Give me a break. If Israel is really just ""te...",reddit.com,1153237661,1
4,limukala,"Think about it dumbass, they ARE the bad guys ...",reddit.com,1153239253,-1
5,limukala,Maybe the messages the children were writing w...,reddit.com,1153242989,0
6,limukala,"""How close do you think people are now to the ...",reddit.com,1153248046,3
7,limukala,"No, Everest in most definitely NOT the highest...",reddit.com,1153248666,2
8,limukala,"uh, not to nitpick, but Sardinia is not in Gre...",reddit.com,1153249454,7
9,limukala,Defacing or altering currency is only illegal ...,reddit.com,1153289157,2


In [413]:
#Group data s.t. authors/bodies are grouped together
sfp_df = sfp_df.groupby('author')['body'].apply(lambda x: "%s" % ' '.join(x)).reset_index()
#This should result in 1 combined string per person (each person posts one "super post" made up of all their posts)

#Import and utilize a function from Jack's script which roughly sanitizes input strings
from fighting_words_py3 import basic_sanitize 
sfp_df['body'] = sfp_df['body'].apply(lambda x: basic_sanitize(x).split()) 

#Replace all strings which start with "http" with a marker for hyperlinks
sfp_df['body'] = sfp_df['body'].apply(lambda x: [i if i[0:4] != 'http' else '<HYPERLINK>' for i in x])
sfp_df.head(10)

,author,body
0,-Gaka-,"[at, the, start, of, this, election, cycle, sp..."
1,-GheeButtersnaps-,"[better, her, than, the, rest, theyre, all, co..."
2,-MOPPET-,"[i, missed, the, registration, deadline, for, ..."
3,-SHMOHAWK-,"[yeah, please, fill, me, in, wonder, what, the..."
4,-Tesserex-,"[are, there, common, ranges, of, ips, in, the,..."
5,-chia-,"[just, common, sense, from, bernie, here, no, ..."
6,0116316,"[i, dont, have, a, choice, my, work, leaves, c..."
7,0826,"[me, too, i, was, starting, to, think, this, w..."
8,0beatbryce,"[ummm, have, you, read, the, comments, people,..."
9,0m3r7a,"[never, thought, of, making, bernie, in, xcom,..."


In [414]:
#Export data
sfp_df.to_csv('./data/language_models/sfp_preproc_langmodel.csv', index=False)

In [412]:
sfp_df.head()

,author,body,subreddit,created_utc,score
0,Jesuslordofporn,"If that is the case, I wonder if he would make...",SandersForPresident,1454297222,0
1,2gurl2gurl,The key through getting legislation through Co...,SandersForPresident,1454302379,5
2,2gurl2gurl,They win because we sit at home unless we are ...,SandersForPresident,1454302485,2
3,jrrl,"Or, better, spend those 30 minutes meeting oth...",SandersForPresident,1454305172,11
4,Smittyblack,Does he have a guy coming in early for himself...,SandersForPresident,1454306535,6
